# Drake Passsage Transport

<i>From Stephen Griffies -- CM4X Development<br>https://github.com/StephenGriffies/CM4X</i>

This notebook plots the eastward transport through the Drake Passage.

### MAR Configuration

In [1]:
import os
import datetime
import doralite
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import doralite
import glob
import subprocess
import momlevel
import itertools

from datetime import timedelta

print("numpy version  : ", np.__version__)
print("pandas version : ", pd.__version__)
print("xarray version : ", xr.__version__)

config = {
    "startyr": "1958",
    "endyr": "2022",
    "dora_id": "odiv-384",
    "pathPP": "",
}

for k, v in config.items():
    config[k] = (
        os.environ[f"MAR_{k.upper()}"]
        if f"MAR_{k.upper()}" in os.environ.keys()
        else v
    )

numpy version  :  1.26.4
pandas version :  2.2.2
xarray version :  2024.5.0


MAR will pass for environment variables to the script when running via the web engine:

* `MAR_STARTYR`: Beginning year of analysis from model
* `MAR_ENDYR`: Ending year of analysis from model
* `MAR_DORA_ID`: The experiment ID in the database
* `MAR_PATHPP`: The top-level path to the post-processing experiment directory of the experiment

The block below will use values passed in by Dora but default to the values defined above in `config`. This is useful for interactive usage and debugging.

If executed from Dora, there will also be a `DORA_EXECUTE` variable that is set.

In [2]:
# Define some local variables. These are taken from the doralite object
# or they can be defined locally

# 1. List of all experiments to consider
experiments = [doralite.dora_metadata(x) for x in config["dora_id"].split(",")]

# Define start and end years
start = int(config["startyr"])
end = int(config["endyr"])

print(start,end)

1958 2022


### Load Files

In [3]:
def is_in_range(file,start,end):
    start = int(start)
    end = int(end)
    target = set(list(range(start,end+1)))
    fname = os.path.basename(file)
    times = fname.split(".")[1]
    times = times.split("-")
    times = [int(x[0:4]) for x in times]
    candidate = set(list(range(times[0],times[1]+1)))
    return len(candidate.intersection(target)) > 0

In [4]:
datafiles = []
staticfiles = []

for experiment in experiments:
    _pathPP = experiment["pathPP"]
    
    component = "ocean_Drake_Passage"
    static = f"{component}/{component}.static.nc"
    varname = "umo"
    
    chunk = "5yr"
    filelist = sorted(glob.glob(f"{_pathPP}{component}/ts/monthly/{chunk}/{component}.*.{varname}.nc"))
    
    filelist = [x for x in filelist if is_in_range(x,start,end)]
    _staticfile = f"{_pathPP}/{static}"
    
    staticfiles.append(_staticfile)
    datafiles.append(filelist)

In [5]:
allfiles = sorted(list(itertools.chain(*datafiles)) + staticfiles)
_ = [print(x) for x in allfiles]

/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp//ocean_Drake_Passage/ocean_Drake_Passage.static.nc
/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp/ocean_Drake_Passage/ts/monthly/5yr/ocean_Drake_Passage.195801-196212.umo.nc
/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp/ocean_Drake_Passage/ts/monthly/5yr/ocean_Drake_Passage.196301-196712.umo.nc
/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp/ocean_Drake_Passage/ts/monthly/5yr/ocean_Drake_Passage.196801-197212.umo.nc
/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp/ocean_Drake_Passage/ts/monthly/5yr/ocean_Drake_Passage.197301-197712.umo.nc
/archive/Brandon.Reichl/fre_om5/FMS2023.01_om5_20240508/om5_b04_fixICbug/gfdl.ncrc5-intel23-prod/pp/ocean_Drake_Passage/ts/monthly/5yr/ocean_Drak

In [ ]:
if not "DORA_EXECUTE" in os.environ.keys():
    print("Calling dmget on files ...")
    cmd = ["dmget"]+allfiles
    _ = subprocess.check_output(cmd)

Calling dmget on files ...


### Calculations and Plots from CM4X Notebook

In [ ]:
# Open dataset
ds = xr.open_mfdataset(filelist, use_cftime=True)

# Sum transport and convert to Sv
umo = ds['umo'].sum(dim=['yh_sub02','z_l'])/1e9

# Drop singleton dimension
umo = umo.squeeze()

# Compute the time mean, taking into account the number of days in a month
# (momlevel utility appropriately handles gregorian and noleap calendars)
umo = momlevel.util.annual_average(umo)

In [ ]:
fig, axis = plt.subplots(ncols=1, nrows=1, figsize=(8,4), dpi=100)

for n,exp in enumerate(experiments):

    expName = exp["expName"]
    
    # Open dataset
    ds = xr.open_mfdataset(datafiles[n], use_cftime=True)  
    # Sum transport and convert to Sv
    umo = ds['umo'].sum(dim=['yh_sub02','z_l'])/1e9
    # Drop singleton dimension
    umo = umo.squeeze()
    # Compute the time mean, taking into account the number of days in a month
    # (momlevel utility appropriately handles gregorian and noleap calendars)
    umo = momlevel.util.annual_average(umo)

    umo.plot.line(ax=axis, label=expName,linestyle='solid', linewidth=1.5)
    
axis.legend(loc='lower center',fontsize=10)
axis.set_ylim([115,175])
axis.set_ylabel('Sv ($10^{9}$kg/s)', fontsize=12)
axis.set_xlabel('year', fontsize=12)
axis.grid(linewidth=1, linestyle="dotted")

axis.set_title('Drake Passage Eastward Mass Transport (Sv)', fontsize=14)
